In [1]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests

In [73]:
class HelperFunctions():
    def getYears(self,quarters): #go through each file name and find the year of filing
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    def getUniqueYears(self,quarters):
        return set(self.getYears(quarters))
    
    def getExcelSheetsPostDate(self,Excel): #use IncomeExcel dictionary to get all dates and only keep where date is later than year specified 
        quarters = Excel.keys()
        fileDates = self.getYears(quarters) 
        files = [file for file, date in zip(Excel, fileDates) if date>self.fromDate or (date==self.fromDate and "Year Ended" in file)]
        Excel = {file:Excel[file] for file in files}
        return Excel
        
    def reorderQuarters(self,compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        cols = [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"] if q + " " + str(year) in quarters]
        compiledStatement = compiledStatement[["label"] + cols]
        return compiledStatement
        
    def getQuarters(self, compiledStatement):
        return list(compiledStatement.columns[1:])
    
    
    def cleanup_label(self, label):
        new_label = label.lower()
        new_label = new_label.replace("gain","loss").replace("decrease","increase")
        new_label = re.sub("\s\([\w\W]+?\)","",new_label).replace("  "," ").replace("—"," - ").strip()
        new_label = re.sub(" \$|\s*\d+\,*", "", new_label) 
        return new_label
    

In [78]:
class Income(HelperFunctions):
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Income Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Income Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.performIncomeMath()
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)
        self.writeExcel()
        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        
        Excel.save()
        
    def addData(self, Excel):
        compiledStatement = pd.DataFrame(columns=["label"])
        
        for quarter,sheet in list(Excel.items()):
            quarter =  re.sub("\s\([\w\W]+?\)","",quarter) #remove the parenthesis ie. (2021) from the quarter
            sheet.columns = ["label",quarter]
            sheet.label = sheet.label.apply(self.cleanup_label)
            
            #sheet = sheet.dropna(axis=0, how="any")
           
            #sheet = sheet[~sheet[quarter].astype("string").str.contains("[a-zA-Z]",regex=True)]

            sheeti = statementi = 0
            newStatement = pd.DataFrame(columns=["label"])
            #merge the current finalstatement and new sheet into one statement 
            #go through each row of the two statements and keep track of the indexes of each statement
            #continue until one of either the compiled statement or sheet ends
            while sheeti < len(sheet) and statementi < len(compiledStatement):
              
                sheet_label = sheet.iloc[sheeti].label
                statement_label = compiledStatement.iloc[statementi].label
                newRowStatement = compiledStatement.iloc[statementi:statementi+1].reset_index(drop=True) #grab the row and drop the index so that they merge
                newRowSheet = sheet.iloc[sheeti:sheeti+1].reset_index(drop=True)
                
                #check if the labels are the same 
                if statement_label == sheet_label:
                    #if labels are the same then create a new row by concating the rows
                    newRowSheet = newRowSheet.drop(columns="label")
                    newRow = pd.concat([newRowStatement,newRowSheet],axis=1)
                    
                    #concat the newRow to the newstatement
                    newStatement = pd.concat([newStatement, newRow])
                    #move on to the next row
                    sheeti+=1
                    statementi+=1
                    
                else:
                    maxi = min(sheeti+4,len(sheet))
                    
                    #next, before I append the final statement label, I want to check if the statement label corresponds to the next label for the sheet
                    #if it corresponds, then I will move the sheet up instead
                    if statement_label in list(sheet.iloc[sheeti:maxi].label): 
                        newStatement = pd.concat([newStatement,newRowSheet]) #concat the sheet row to the new statement
                        sheeti += 1
                        continue
                    
                    #if the two labels don't match, default is to add the row of the final statment
                    newStatement = pd.concat([newStatement,newRowStatement])
                    statementi += 1
                    
            #if either the finalstatement or the new sheet hasn't finished, then add the end of the statement to the new one
            newStatement = pd.concat([newStatement,sheet.iloc[sheeti:len(sheet)]])
            newStatement = pd.concat([newStatement,compiledStatement.iloc[statementi:len(compiledStatement)]])
            
            compiledStatement = newStatement
            
        compiledStatement = compiledStatement.reset_index(drop=True)
        return compiledStatement
    
    def performIncomeMath(self):
        quarters = self.getQuarters(self.compiledStatement)
        years = self.getUniqueYears(quarters)
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                self.compiledStatement["Q4 "+year] = self.compiledStatement["Year Ended "+year] - self.compiledStatement["Q3 "+year]
                self.compiledStatement.loc[self.compiledStatement.label.str.startswith("weighted-average"),"Q4 "+year] = self.compiledStatement.loc[self.compiledStatement.label.str.startswith("weighted-average"),"Year Ended "+year]


            

In [82]:
income = Income("WMT",2015)
income.compiledStatement



,label,Q3 2018,Q4 2018,Year Ended 2018,Q1 2019,Q2 2019,Q3 2019,Q4 2019,Year Ended 2019,Q1 2020,...,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022,Year Ended 2022,Q1 2023
0,revenues:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,net sales,122136.00,373625.00,495761.00,121630.00,127059.00,123897.00,386432.00,510329.00,122949.00,...,136824.00,133752.00,421481.00,555233.00,137159.00,139871.00,139207.00,428555.00,567762.00,140288.00
2,membership and other income,1043.00,3539.00,4582.00,1060.00,969.00,997.00,3079.00,4076.00,976.00,...,918.00,956.00,2962.00,3918.00,1151.00,1177.00,1318.00,3674.00,4992.00,1281.00
3,total revenues,123179.00,377164.00,500343.00,122690.00,128028.00,124894.00,389511.00,514405.00,123925.00,...,137742.00,134708.00,424443.00,559151.00,138310.00,141048.00,140525.00,432229.00,572754.00,141569.00
4,costs and expenses:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cost of sales,91547.00,281849.00,373396.00,91707.00,95571.00,93116.00,292185.00,385301.00,93034.00,...,102689.00,100339.00,319976.00,420315.00,103272.00,105183.00,105023.00,323977.00,429000.00,106847.00
6,"operating, selling, general and administrative...",26868.00,79642.00,106510.00,25829.00,26707.00,26792.00,80355.00,107147.00,25946.00,...,28994.00,28591.00,87697.00,116288.00,28129.00,28511.00,29710.00,88102.00,117812.00,29404.00
7,operating income,4764.00,15673.00,20437.00,5154.00,5750.00,4986.00,16971.00,21957.00,4945.00,...,6059.00,5778.00,16770.00,22548.00,6909.00,7354.00,5792.00,20150.00,25942.00,5318.00
8,interest:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,debt,502.00,1476.00,1978.00,437.00,460.00,501.00,1474.00,1975.00,588.00,...,577.00,455.00,1521.00,1976.00,481.00,437.00,408.00,1266.00,1674.00,372.00


In [88]:
class Cashflow(HelperFunctions):
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Cashflow Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Cashflow Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)

        self.compiledStatement = self.reorderQuarters(self.compiledStatement)
        self.writeExcel()
        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        
        Excel.save()
        
    def addData(self, Excel):
        compiledStatement = pd.DataFrame(columns=["label"])
        
        for quarter,sheet in list(Excel.items()):
            quarter =  re.sub("\s\([\w\W]+?\)","",quarter) #remove the parenthesis ie. (2021) from the quarter
            sheet.columns = ["label",quarter]
            sheet.label = sheet.label.apply(self.cleanup_label)
            
            #sheet = sheet.dropna(axis=0, how="any")
           
            #sheet = sheet[~sheet[quarter].astype("string").str.contains("[a-zA-Z]",regex=True)]

            sheeti = statementi = 0
            newStatement = pd.DataFrame(columns=["label"])
            #merge the current finalstatement and new sheet into one statement 
            #go through each row of the two statements and keep track of the indexes of each statement
            #continue until one of either the compiled statement or sheet ends
            while sheeti < len(sheet) and statementi < len(compiledStatement):
              
                sheet_label = sheet.iloc[sheeti].label
                statement_label = compiledStatement.iloc[statementi].label
                newRowStatement = compiledStatement.iloc[statementi:statementi+1].reset_index(drop=True) #grab the row and drop the index so that they merge
                newRowSheet = sheet.iloc[sheeti:sheeti+1].reset_index(drop=True)
                
                #check if the labels are the same 
                if statement_label == sheet_label:
                    #if labels are the same then create a new row by concating the rows
                    newRowSheet = newRowSheet.drop(columns="label")
                    newRow = pd.concat([newRowStatement,newRowSheet],axis=1)
                    
                    #concat the newRow to the newstatement
                    newStatement = pd.concat([newStatement, newRow])
                    #move on to the next row
                    sheeti+=1
                    statementi+=1
                    
                else:
                    maxi = min(sheeti+4,len(sheet))
                    
                    #next, before I append the final statement label, I want to check if the statement label corresponds to the next label for the sheet
                    #if it corresponds, then I will move the sheet up instead
                    if statement_label in list(sheet.iloc[sheeti:maxi].label): 
                        newStatement = pd.concat([newStatement,newRowSheet]) #concat the sheet row to the new statement
                        sheeti += 1
                        continue
                    
                    #if the two labels don't match, default is to add the row of the final statment
                    newStatement = pd.concat([newStatement,newRowStatement])
                    statementi += 1
                    
            #if either the finalstatement or the new sheet hasn't finished, then add the end of the statement to the new one
            newStatement = pd.concat([newStatement,sheet.iloc[sheeti:len(sheet)]])
            newStatement = pd.concat([newStatement,compiledStatement.iloc[statementi:len(compiledStatement)]])
            
            compiledStatement = newStatement
            
        compiledStatement = compiledStatement.reset_index(drop=True)
        return compiledStatement
    
   
            

In [94]:
cashflow = Cashflow("WMT",2020)
cashflow.compiledStatement



,label,Year Ended 2020,Q1 2021,Q2 2021,Q3 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Year Ended 2022,Q1 2023
0,cash flows from operating activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consolidated net income,15201.0,4074.0,10513.0,15714.0,13706.0,2811.0,7175.0,10307.0,13940.0,2103.0
2,adjustments to reconcile income from continuin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adjustments to reconcile consolidated net inco...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,depreciation and amortization,10987.0,2791.0,5562.0,8333.0,11152.0,2661.0,5302.0,7952.0,10658.0,2680.0
5,unrealized and losses,-1886.0,-783.0,-4006.0,-6883.0,NaN,NaN,NaN,NaN,NaN,NaN
6,(losss) and losses for disposal of business op...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,net unrealized and realized and losses,NaN,NaN,NaN,NaN,-8589.0,2077.0,3019.0,1831.0,2440.0,1989.0
8,losses on disposal of business operations,NaN,NaN,NaN,NaN,8401.0,433.0,433.0,433.0,433.0,0.0
9,asda pension contribution,-1036.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
